# **Tạo và lưu Span Adaptation Weights cho các argument (ARG)**

## Mục đích

- Sử dụng mô hình SRL đã fine-tuned để tính trọng số content pooling cho từng token trong span của mỗi argument (ARG-0, ARG-1, ...).
- Kết hợp với các adaptation vectors đã được trích xuất trước đó cho từng token.
- Lưu lại các trọng số này theo từng:
  - câu (sentence_i),
  - argument (ARG-x),
  - layer (layer_0, layer_1, ...),
  dưới dạng file `.pt` riêng để phục vụ cho các bước xử lý downstream sau này (ví dụ: tạo vector span, phân loại, tính tương đồng, v.v.).


## Đầu vào

1. Mô hình và tokenizer
   - Mô hình SRL đã fine-tuned:
     - Được load từ `Finetuned_Models/biobert-srl-best-model`.
     - Dùng `AutoModelForTokenClassification`.
   - Tokenizer tương ứng:
     - Dùng `AutoTokenizer.from_pretrained(final_model_path)`.

2. Dữ liệu câu và argument (dạng JSON)
   - Nằm trong các thư mục:
     - `Clean_Dataset/Corpus/Split_GramVar/Train`
     - `Clean_Dataset/Corpus/Split_GramVar/Test`
     - `Clean_Dataset/Corpus/Split_ParaVE/Train`
     - `Clean_Dataset/Corpus/Split_ParaVE/Test`
   - Mỗi file JSON chứa danh sách các phần tử, mỗi phần tử có dạng:
     - `text`: câu đầy đủ (string).
     - `arguments`: dict các argument, ví dụ:
       - `{"ARG-0": "...", "ARG-1": "...", ...}`.

3. Adaptation vectors cho từng câu
   - Nằm trong các thư mục:
     - `adaptation_vectors_train_gramvar_pt_aligned`
     - `adaptation_vectors_test_gramvar_pt_aligned`
     - `adaptation_vectors_train_parave_pt_aligned`
     - `adaptation_vectors_test_parave_pt_aligned`
   - Mỗi câu tương ứng với một file:
     - `sentence_i.pt` bên trong thư mục động từ, ví dụ:
       - `.../adaptation_vectors_train_gramvar_pt_aligned/<verb_name>/sentence_0.pt`
   - Cấu trúc mỗi file `.pt`:
     - Dict: `{ "layer_0": tensor[num_tokens, hidden_dim], "layer_1": tensor[...], ... }`.

4. Thiết bị tính toán
   - Tự động chọn:
     - GPU (CUDA) nếu khả dụng.
     - CPU nếu không có GPU.

## Đầu ra

1. Các file trọng số span adaptation cho từng argument
   - Được lưu trong các thư mục:
     - `span_adaptation_weights_train_gramvar`
     - `span_adaptation_weights_test_gramvar`
     - `span_adaptation_weights_train_parave`
     - `span_adaptation_weights_test_parave`
   - Bên trong mỗi thư mục trên là các thư mục con theo động từ `verb_name`.

2. Cấu trúc từng file trọng số
   - Tên file:
     - `sentence_<i>_<ARG_x>.pt`, ví dụ:
       - `sentence_0_ARG_0.pt`
       - `sentence_3_ARG_1.pt`
   - Nội dung file:
     - Dict: `{ "layer_0": tensor[num_tokens_in_span], "layer_1": tensor[...], ... }`
     - Mỗi tensor là các trọng số `w_k` tương ứng với từng token trong span của argument tại layer đó.

## Quy trình xử lý

1. Thiết lập mô hình và đường dẫn
   - Chọn `device = cuda` nếu có GPU, ngược lại dùng `cpu`.
   - Khai báo `drive_base_path` trỏ đến thư mục gốc trên Google Drive.
   - Load tokenizer và mô hình SRL fine-tuned từ `final_model_path`.
   - Đặt mô hình sang chế độ `eval()`.

2. Định nghĩa danh sách bộ dữ liệu cần xử lý
   - `datasets_to_process` là list các tuple:
     - `(name, data_dir, av_input_dir, output_dir)`
   - Mỗi tuple tương ứng với:
     - Tên bộ dữ liệu (Train/Test + GramVar/ParaVE).
     - Thư mục dữ liệu JSON gốc.
     - Thư mục chứa adaptation vectors.
     - Thư mục sẽ lưu span weights.

3. Hàm `find_sub_list(sl, l)`
   - Mục đích:
     - Tìm vị trí bắt đầu và kết thúc (`start_idx`, `end_idx`) của list con `sl` trong list `l`.
   - Ứng dụng:
     - Tìm vị trí span của argument (tokens) trong toàn bộ câu (tokens).

4. Hàm `create_span_weights(sentence_text, arguments, token_av_data, model, tokenizer)`
   - Tokenize câu `sentence_text` để sinh `full_tokens`.
   - Chạy mô hình:
     - Lấy `logits` và `predictions` (nhãn dự đoán cho từng token).
   - Lấy `classifier_weights` từ lớp cuối cùng của mô hình (`model.classifier.weight`).

   - Với mỗi argument trong `arguments`:
     1. Chỉ xét các key bắt đầu bằng `"ARG"`.
     2. Tokenize nội dung argument → `arg_tokens`.
     3. Dùng `find_sub_list(arg_tokens, full_tokens)` để tìm span trong câu.
     4. Điều chỉnh chỉ số token để bỏ qua `[CLS]`:
        - `start_idx += 1`, `end_idx += 1`.
     5. Nếu không tìm được span hợp lệ thì bỏ qua argument.

   - Với mỗi layer trong `token_av_data`:
     1. Lấy adaptation vectors của toàn bộ câu ở layer đó.
     2. Cắt ra phần tương ứng với span argument:
        - `span_token_avs = layer_av_tensor[start_idx : end_idx + 1]`.
     3. Lấy nhãn dự đoán tương ứng cho các token trong span:
        - `span_token_preds = predictions[start_idx : end_idx + 1]`.

     4. Tính các hệ số tạm `a_k`:
        - Với mỗi token trong span:
          - Lấy vector thích nghi `adap_vec_tk`.
          - Lấy vector trọng số classifier `w_r_star` tương ứng với nhãn.
          - Tính:
            - `dot_product = adap_vec_tk . w_r_star`
            - `norm_w = ||w_r_star||`
            - `a_k = max(0, dot_product / (norm_w + 1e-8))`
     5. Tính tổng:
        - `sum_a_k = sum(a_k_list)`.

     6. Chuẩn hóa thành trọng số `w_k`:
        - Trường hợp `sum_a_k > 1e-8`:
          - `w_k = a_k / (sum_a_k + 1e-8)` cho từng token.
        - Trường hợp `sum_a_k == 0` và có ít nhất một token:
          - Dùng average:
            - mọi `w_k = 1 / num_tokens`.
        - Trường hợp không có token:
          - Trả về tensor rỗng.

     7. Lưu trọng số per-layer cho argument:
        - `span_weights_per_layer[layer_name] = w_k_tensor`.

   - Hàm trả về:
     - `span_weights_all_args`:
       - Dict: `{ "ARG-0": {...}, "ARG-1": {...}, ... }`
       - Mỗi ARG chứa dict `layer_name → tensor(weights)`.

5. Vòng lặp chính trong `if __name__ == "__main__":`
   - Với mỗi bộ dữ liệu trong `datasets_to_process`:
     1. Tạo thư mục output nếu chưa tồn tại.
     2. Liệt kê toàn bộ file JSON trong `data_dir`.
     3. Với mỗi file JSON:
        - Suy ra `verb_name` từ tên file (loại bỏ `_train_set.json` hoặc `_test_set.json`).
        - Tạo thư mục `output_verb_dir` tương ứng với động từ.
        - Đọc toàn bộ nội dung JSON:
          - Dạng list, mỗi phần tử là một câu.

     4. Với mỗi câu (theo index `i`):
        - Lấy `sentence_text` và `arguments`.
        - Nếu thiếu 1 trong 2 → bỏ qua.
        - Xác định đường dẫn adaptation vector:
          - `av_path = os.path.join(av_input_dir, verb_name, f"sentence_{i}.pt")`
        - Load `token_av_data` bằng `torch.load(...)`.
        - Gọi `create_span_weights(...)` để tính trọng số cho từng ARG.

     5. Lưu kết quả:
        - Với mỗi `arg_label` trong `span_weights_data_per_arg`:
          - Thay `-` bằng `_` trong `arg_label` để tên file an toàn.
          - Tạo tên file:
            - `sentence_<i>_<safe_arg_label>.pt`
          - Dùng `torch.save(weights_for_this_arg, output_path)` để lưu dict trọng số.

     6. Xử lý lỗi:
        - Nếu không có file adaptation vectors tương ứng (`FileNotFoundError`):
          - Bỏ qua câu đó.
        - Các lỗi khác:
          - In thông báo lỗi kèm chỉ số câu và động từ.

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
import json
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification

# THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH
# Tự động chọn GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Tải lại mô hình fine-tuned để lấy trọng số classifier và chạy dự đoán
print("Đang tải lại mô hình Fine-tuned...")
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')
tokenizer = AutoTokenizer.from_pretrained(final_model_path)
model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path).to(device)
model_ft.eval() # Chuyển sang chế độ đánh giá
print(" -> Tải mô hình thành công!")

# --- DANH SÁCH CÁC BỘ DỮ LIỆU CẦN XỬ LÝ ---
datasets_to_process = [
    # --- Dữ liệu Train ---
    (
        "Train_GramVar",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train'), # input_data_dir
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_gramvar_pt_aligned'), # av_input_dir (token vectors)
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Weight/span_adaptation_weights_train_gramvar') # lưu weights
    ),
    (
        "Train_ParaVE",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_parave_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Weight/span_adaptation_weights_train_parave')
    ),
    # --- Dữ liệu Test ---
    (
        "Test_GramVar",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test'),
        os.path.join(drive_base_path, 'Adaptation Vector/Test/adaptation_vectors_test_gramvar_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Weight/span_adaptation_weights_test_gramvar')
    ),
    (
        "Test_ParaVE",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_test_parave_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Weight/span_adaptation_weights_test_parave')
    ),
]

# CÁC HÀM XỬ LÝ

def find_sub_list(sl, l):
    """Tìm vị trí của một list con (sl) trong một list lớn (l)."""
    sll = len(sl)
    for ind in (i for i, e in enumerate(l) if e == sl[0]):
        if l[ind:ind+sll] == sl:
            return ind, ind + sll - 1
    return -1, -1

# Tên hàm và chức năng: chỉ tạo và trả về TRỌNG SỐ
def create_span_weights(sentence_text, arguments, token_av_data, model, tokenizer):
    """
    Chỉ tạo và trả về trọng số content pooling
    cho tất cả các argument trong một câu.
    """
    full_tokens = tokenizer.tokenize(sentence_text)

    with torch.no_grad():
        inputs = tokenizer(sentence_text, return_tensors="pt").to(device)
        logits = model(**inputs).logits
        predictions = torch.argmax(logits, dim=2)[0].cpu().numpy()

    classifier_weights = model.classifier.weight.detach().cpu()

    span_weights_all_args = {}

    for arg_label, arg_text in arguments.items():
        if not arg_label.startswith('ARG') or not arg_text:
            continue

        arg_tokens = tokenizer.tokenize(arg_text)
        start_idx, end_idx = find_sub_list(arg_tokens, full_tokens)

        # +1 vì ta bỏ qua [CLS]
        start_idx += 1
        end_idx += 1

        if start_idx == 0: # (0-1)+1
            continue

        span_weights_per_layer = {}
        for layer_name, layer_av_tensor in token_av_data.items():
            layer_av_tensor = layer_av_tensor.cpu()

            # Không cần vector 'begin' và 'end'
            # adap_vec_begin = layer_av_tensor[start_idx]
            # adap_vec_end = layer_av_tensor[end_idx]

            # 2. Lấy các vector Content
            span_token_avs = layer_av_tensor[start_idx : end_idx + 1]
            span_token_preds = predictions[start_idx : end_idx + 1]

            # 3. Tính trọng số
            a_k_list = []
            for i in range(len(span_token_avs)):
                adap_vec_tk = span_token_avs[i]
                pred_label_id = span_token_preds[i]
                w_r_star = classifier_weights[pred_label_id]

                dot_product = torch.dot(adap_vec_tk, w_r_star)
                norm_w = torch.linalg.norm(w_r_star)
                a_k = torch.max(torch.tensor(0.0), dot_product / (norm_w + 1e-8))
                a_k_list.append(a_k)

            sum_a_k = torch.sum(torch.stack(a_k_list))

            # --- PHẦN TÍNH TOÁN VÀ LƯU TRỌNG SỐ ---
            w_k_list = []
            num_tokens = len(span_token_avs)

            if sum_a_k > 1e-8:
                # TH1: Tính weighted average
                sum_a_k_plus_eps = sum_a_k + 1e-8
                for i in range(num_tokens):
                    w_k = a_k_list[i] / sum_a_k_plus_eps
                    w_k_list.append(w_k)
            elif num_tokens > 0:
                # TH2: Fallback về average nếu sum_a_k = 0
                w_k = torch.tensor(1.0 / num_tokens)
                for i in range(num_tokens):
                    w_k_list.append(w_k)
            # TH3: num_tokens = 0, w_k_list = []

            # 5. Tạo tensor trọng số
            if w_k_list:
                w_k_tensor = torch.stack(w_k_list).cpu()
            else:
                w_k_tensor = torch.empty(0, dtype=torch.float32).cpu()

            # Chỉ lưu trọng số
            # Lưu ý: Cấu trúc lưu là dict[layer_name] = tensor_trọng_số
            span_weights_per_layer[layer_name] = w_k_tensor

        span_weights_all_args[arg_label] = span_weights_per_layer

    return span_weights_all_args

# THỰC THI CHƯƠNG TRÌNH

if __name__ == "__main__":
    print("\n--- Bắt đầu quá trình tạo Span Adaptation **WEIGHTS** ---")

    for name, data_dir, av_input_dir, output_dir in datasets_to_process:
        print(f"\n=================================================")
        print(f"BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: {name}")
        print(f" -> Sẽ lưu trọng số vào: {output_dir}")
        print(f"=================================================")

        os.makedirs(output_dir, exist_ok=True)
        json_files = glob.glob(os.path.join(data_dir, '*.json'))

        if not json_files:
            print(f"CẢNH BÁO: Không tìm thấy file dữ liệu gốc nào trong '{data_dir}'. Bỏ qua.")
            continue

        pbar = tqdm(total=len(json_files), desc=f"Xử lý {name}", unit="file")

        for json_file in json_files:
            verb_name = os.path.basename(json_file).replace('_test_set.json', '').replace('_train_set.json', '')

            with open(json_file, 'r', encoding='utf-8') as f:
                original_data = json.load(f)

            output_verb_dir = os.path.join(output_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            for i, item in enumerate(original_data):
                sentence_text = item.get('text')
                arguments = item.get('arguments')

                if not sentence_text or not arguments:
                    continue

                try:
                    av_path = os.path.join(av_input_dir, verb_name, f"sentence_{i}.pt")
                    token_av_data = torch.load(av_path, map_location='cpu')

                    # span_weights_data_per_arg là dict, ví dụ: {"ARG-0": {"layer_0":...}, "ARG-1": ...}
                    span_weights_data_per_arg = create_span_weights(sentence_text, arguments, token_av_data, model_ft, tokenizer)

                    # Lặp qua từng argument và lưu thành file riêng
                    if span_weights_data_per_arg:
                        for arg_label, weights_for_this_arg in span_weights_data_per_arg.items():
                            # Tạo tên file mới, ví dụ: sentence_0_ARG-0.pt
                            safe_arg_label = arg_label.replace('-', '_')
                            output_filename = f"sentence_{i}_{safe_arg_label}.pt"
                            output_path = os.path.join(output_verb_dir, output_filename)

                            # Lưu `weights_for_this_arg`
                            # (Đây là dict {"layer_0": tensor, "layer_1": tensor,...})
                            torch.save(weights_for_this_arg, output_path)

                except FileNotFoundError:
                    # pbar.write(f"Không tìm thấy file vector: {av_path}")
                    pass
                except Exception as e:
                    pbar.write(f"Lỗi khi xử lý câu {i}, động từ {verb_name}: {e}")

            pbar.update(1)
        pbar.close()

    print("\n--- QUÁ TRÌNH TẠO SPAN ADAPTATION **WEIGHTS** ĐÃ HOÀN TẤT! ---")

Sử dụng thiết bị: cuda
Đang tải lại mô hình Fine-tuned...
 -> Tải mô hình thành công!

--- Bắt đầu quá trình tạo Span Adaptation **WEIGHTS** (Lưu riêng) ---

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Train_GramVar
 -> Sẽ lưu trọng số vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_weights_train_gramvar


Xử lý Train_GramVar:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Train_ParaVE
 -> Sẽ lưu trọng số vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_weights_train_parave


Xử lý Train_ParaVE:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Test_GramVar
 -> Sẽ lưu trọng số vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_weights_test_gramvar


Xử lý Test_GramVar:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Test_ParaVE
 -> Sẽ lưu trọng số vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_weights_test_parave


Xử lý Test_ParaVE:   0%|          | 0/35 [00:00<?, ?file/s]


--- QUÁ TRÌNH TẠO SPAN ADAPTATION **WEIGHTS** ĐÃ HOÀN TẤT! ---
